In [5]:
%autoawait
import time
import asyncio

In [6]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


In [7]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

# connect 3 agents

## 1. connect to SP

In [8]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


In [9]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection e04213f6-b268-42b0-abc5-6279027688c2 in State invitation
Connection ID e04213f6-b268-42b0-abc5-6279027688c2
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '51faa42a-8769-450c-815e-3d1b022134a2', 'recipientKeys': ['6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}
Connection Handler Called
Connection e04213f6-b268-42b0-abc5-6279027688c2 in State request


### 2. switch to SP 

### 3.

In [10]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection e04213f6-b268-42b0-abc5-6279027688c2 in State response
ACCEPT REQUEST
{'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:11.853478Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:18:48.577736Z', 'my_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'their_did': 'MBSF9UmbuFnAftoJx2pw1j', 'state': 'response', 'their_label': 'Carol', 'connection_id': 'e04213f6-b268-42b0-abc5-6279027688c2', 'rfc23_state': 'response-sent', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
state response


In [11]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '84dc281b-0f21-4b9d-80c2-9d1e73f6a888'}
Connection Handler Called
Connection e04213f6-b268-42b0-abc5-6279027688c2 in State active


In [12]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
sp_connection = connection["connection_id"]
print(sp_connection)

{'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:13.425759Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:18:48.577736Z', 'my_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'their_did': 'MBSF9UmbuFnAftoJx2pw1j', 'state': 'active', 'their_label': 'Carol', 'connection_id': 'e04213f6-b268-42b0-abc5-6279027688c2', 'rfc23_state': 'completed', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
Is Active? active
e04213f6-b268-42b0-abc5-6279027688c2


In [13]:
# sp_connection = 'e6557bae-b92b-4ad7-8c55-b91d496f6ce1'

In [14]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'updated_at': '2023-01-03 12:11:59.431606Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:10:47.835278Z', 'my_did': 'XtNBn5VgqaouerjBv9zGiy', 'their_did': 'JwZFj9aDdiL1d4pJSSL3Wj', 'state': 'active', 'their_label': 'Bob', 'connection_id': '3a5b59b0-f221-429b-b8f3-b121e1b7c640', 'rfc23_state': 'completed', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}, {'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:13.425759Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:18:48.577736Z', 'my_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'their_did': 'MBSF9UmbuFnAftoJx2pw1j', 'state': 'active', 'their_label': 'Carol', 'connection_id': 'e04213f6-b268-42b0-abc5-6279027688c2', 'rfc23_state': 'completed', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}]}


In [99]:
# response = await agent_controller.connections.remove_connection('bf329959-1a75-47fb-a602-a7cd9722fa90')

### 4. switch to SP

## 9. connect to MUSIC

In [15]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection bb9dd96d-ef0d-4859-8297-77800b8cc9a6 in State invitation
Connection ID bb9dd96d-ef0d-4859-8297-77800b8cc9a6
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e3a2ca2c-d327-4335-8771-0f36013c848d', 'recipientKeys': ['DHLmYoHNjhVwRLkwf9spNbjmmoAhHthE4D2ADG33ajpc'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}
Connection Handler Called
Connection bb9dd96d-ef0d-4859-8297-77800b8cc9a6 in State request


### 10. switch to MUSIC

### 11.

In [16]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection bb9dd96d-ef0d-4859-8297-77800b8cc9a6 in State response
ACCEPT REQUEST
{'invitation_key': 'DHLmYoHNjhVwRLkwf9spNbjmmoAhHthE4D2ADG33ajpc', 'updated_at': '2023-01-03 12:20:26.186332Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:20:11.801504Z', 'my_did': '9PPzLD1Nhkbqeoq419ZWVC', 'their_did': '7dsM89cArCQEM8LHWsE6WU', 'state': 'response', 'their_label': 'Bob', 'connection_id': 'bb9dd96d-ef0d-4859-8297-77800b8cc9a6', 'rfc23_state': 'response-sent', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
state response


In [17]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'da57c088-490a-41e9-ae5a-677edc9781a2'}
Connection Handler Called
Connection bb9dd96d-ef0d-4859-8297-77800b8cc9a6 in State active


In [18]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

{'invitation_key': 'DHLmYoHNjhVwRLkwf9spNbjmmoAhHthE4D2ADG33ajpc', 'updated_at': '2023-01-03 12:20:28.291631Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:20:11.801504Z', 'my_did': '9PPzLD1Nhkbqeoq419ZWVC', 'their_did': '7dsM89cArCQEM8LHWsE6WU', 'state': 'active', 'their_label': 'Bob', 'connection_id': 'bb9dd96d-ef0d-4859-8297-77800b8cc9a6', 'rfc23_state': 'completed', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
Is Active? active
bb9dd96d-ef0d-4859-8297-77800b8cc9a6


### 12. switch to MUSIC

# exchange DID with SP and MUSIC

## 1.

In [19]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
sp_did = 'None'
music_did = 'None'

my DID PQRXDxdGqQGSZ8z69p4xZP


In [20]:
def messages_handler(payload):
    global sp_did
    global music_did
    connection_id = payload["connection_id"]
    did = payload["content"]
    if did[:3] == 'sp_':
        sp_did = did[4:]
        print('sp_did', sp_did)
    else:
        music_did = did[6:]
        print('music_did', music_did)
        
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, my_did))
    print("Handle message", payload, connection_id)
    
    
    

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


music_did gZj39Y8JGQ5GVQp2y6zFH
Handle message {'connection_id': 'bb9dd96d-ef0d-4859-8297-77800b8cc9a6', 'message_id': '734a5fe0-380c-48bd-9188-318e4542a6d0', 'content': 'music_gZj39Y8JGQ5GVQp2y6zFH', 'state': 'received'} bb9dd96d-ef0d-4859-8297-77800b8cc9a6
sp_did 8HWQruWXrv8s6BBLXm53B
Handle message {'connection_id': 'e04213f6-b268-42b0-abc5-6279027688c2', 'message_id': '81fdcb57-8c32-4766-9496-f49954cc8fa3', 'content': 'sp_X8HWQruWXrv8s6BBLXm53B', 'state': 'received'} e04213f6-b268-42b0-abc5-6279027688c2


In [111]:
message = my_did
response = await agent_controller.messaging.send_message(music_connection, message)
print(response)

{}


# transfer possession

## 2. issue new VC to MUSIC and SP

In [23]:
agent_controller.remove_all_listeners("basicmessages")

In [24]:
revoke = False

In [25]:
def request_handler(payload):
    global revoke
    connection_id = payload["connection_id"]
    if payload['content'] == 'True': 
        revoke = True
    send_task = asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(sp_connection, "revoke: "+str(revoke)))
    print("Handle request", payload, connection_id)


request_listener = {
    "handler": request_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([request_listener], defaults=False)

Subscribing too: basicmessages


Handle request {'connection_id': 'e04213f6-b268-42b0-abc5-6279027688c2', 'message_id': '00940fb0-76b2-43b2-8947-1c3998b567b2', 'content': 'True', 'state': 'received'} e04213f6-b268-42b0-abc5-6279027688c2


## 3. switch to SP

## 4. revoke old VC from MUSIC

In [26]:
print(revoke)

In [37]:
if revoke:
    ## revoke the previous VC
    

## 5. issue new VC to MUSIC

In [27]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

In [28]:
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did=my_did)
print(created_schema)
schema_id = created_schema['schema_ids'][0]
print(schema_id)

## find schema_id

In [29]:
created_cred_def = response = await agent_controller.definitions.search_created(schema_issuer_did=my_did)
print(created_cred_def)
cred_def_id = created_cred_def['credential_definition_ids'][0]
print(cred_def_id)

## find cred_def_id

In [34]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": my_did},
    {"name": "origin", "value": my_did},
    {"name": "owner", "value": sp_did},
    {"name": "time", "value": time},
    {"name": "version", "value": "v1"},
    {"name": "name", "value": "balabababa music"},
    ]
record = await agent_controller.issuer.send_credential(music_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)

Handle Credentials
Credential exchange 923ab876-b172-4b22-8c84-937b5e112e2c, role: issuer, state: offer_sent
Attributes: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 12:33:57'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]
Handle Credentials
Credential exchange ebb434c2-ab5d-47f7-9895-0ebf37021849, role: issuer, state: request_received
Attributes: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 12:31:55'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]
Handle Credentials
Credential exchange ebb434c2-ab5d-47f7-9895-0ebf37021849, role: issuer, state: credential_issued
Attributes: [{'name': 'author', 'value':

In [35]:
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 923ab876-b172-4b22-8c84-937b5e112e2c, role: issuer, state: offer_sent
Handle Credentials
Credential exchange ebb434c2-ab5d-47f7-9895-0ebf37021849, role: issuer, state: credential_acked
Attributes: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 12:31:55'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]


## 6. switch to MUSIC

## 7. define new schema 

In [36]:
# Define you schema name - must be unique on the ledger
schema_name = "possession schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["possession", "time"]


In [37]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

In [38]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

## 8. issue new VC to SP

In [39]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "possession", "value": music_did},
    {"name": "time", "value": time},
]
print(credential_attributes)

[{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]


## 9. switch to SP

## 10.

In [41]:
record = await agent_controller.issuer.send_credential(sp_connection, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
print(record)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

{'state': 'offer_sent', 'auto_remove': False, 'credential_exchange_id': '4401e835-f1c6-413d-9449-7241ba0f45b0', 'updated_at': '2023-01-03 12:36:13.421797Z', 'auto_issue': True, 'role': 'issuer', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'd6b1225a-69d0-4494-8392-103e0a5f1ead', '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]}, 'schema_name': 'possession schema', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:possession schema:0.0.1', 'comment': '', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'schema_version': '0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default'}, 'initiator': 'self', 'trace': True, 'credential_offer': {'schema_

## 11. switch to SP

In [42]:
response = await agent_controller.terminate()
print(response)